# ASL Project

In [1]:
#Import libraries
import numpy as np
import random

# Imports to view data
import cv2
import pandas as pd
from glob import glob

# Visualization
from keras.utils import print_summary
from matplotlib import pyplot as plt

#ML libraries
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint  
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

Using TensorFlow backend.


In [2]:
#Directory paths
TRAIN_DIR = "../Dataset/asl_alphabet_train/asl_alphabet_train"
TEST_DIR = "../Dataset/asl_alphabet_test"
MODEL_DIR = './Model'
MODEL_PATH = MODEL_DIR+"/Model1-xception-2.h5"
MODEL_WEIGHT_PATH = MODEL_DIR+"/Model_Weight1-xception-2.h5"

In [3]:
# Set global variables
TARGET_SIZE = (64, 64)
TARGET_DIMS = (64, 64, 3) # add channel for RGB
CLASSES = 29
VALIDATION_SPLIT = 0.1
BATCH_SIZE = 64
LABELS_NAME = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q',
               'R','S','T','U','V','W','X','Y','Z','del','nothing','space']

## Data Augmentation

In [4]:
#Load Train dataset
train_image_generator = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=VALIDATION_SPLIT
)

validation_image_generator = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
    validation_split=VALIDATION_SPLIT
)

train_generator = train_image_generator.flow_from_directory(TRAIN_DIR, target_size=TARGET_SIZE, batch_size=BATCH_SIZE, shuffle=True, subset="training")
val_generator = validation_image_generator.flow_from_directory(TRAIN_DIR, target_size=TARGET_SIZE, batch_size=BATCH_SIZE, subset="validation")

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


## Model

In [5]:
#Define Model Xception Model
def Xception_model_build():
    from keras.applications.xception import Xception
    from keras.layers import Input

    input_tensor = Input(shape=TARGET_DIMS)
    model = Xception(input_tensor = input_tensor, weights=None, include_top=True, classes= CLASSES)
    return model

In [6]:
from keras.models import Model
base_model = Xception_model_build()
x = base_model.output
#x = (Dense(CLASSES, activation='softmax'))(x)
model = Model(inputs=base_model.input, outputs=x)

#Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
model.summary()
model.save(MODEL_PATH)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 31, 31, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 31, 31, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 31, 31, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [9]:
#Checkpointer to save the best models
checkpointer = ModelCheckpoint(filepath=MODEL_WEIGHT_PATH, 
                               verbose=1, save_best_only=True)

steps_per_epoch = int( np.ceil(len(train_generator)*2 / BATCH_SIZE) )
validation_steps = int( np.ceil(len(val_generator)*2 / BATCH_SIZE) )

model.fit_generator(train_generator, validation_data=val_generator, 
                    steps_per_epoch =  steps_per_epoch,
                    validation_steps = validation_steps,
                    epochs=20, callbacks=[checkpointer], verbose=1)

Epoch 1/20
39/39 [==============================] - 297s 8s/step - loss: 3.6356 - acc: 0.0505 - val_loss: 10.8390 - val_acc: 0.0594

Epoch 00001: val_loss improved from inf to 10.83900, saving model to ./Model/Model_Weight1-xception-2.h5
Epoch 2/20
39/39 [==============================] - 295s 8s/step - loss: 3.3484 - acc: 0.0561 - val_loss: 6.0564 - val_acc: 0.0750

Epoch 00002: val_loss improved from 10.83900 to 6.05639, saving model to ./Model/Model_Weight1-xception-2.h5
Epoch 3/20
39/39 [==============================] - 293s 8s/step - loss: 3.2993 - acc: 0.0733 - val_loss: 7.1521 - val_acc: 0.0563

Epoch 00003: val_loss did not improve from 6.05639
Epoch 4/20
39/39 [==============================] - 294s 8s/step - loss: 3.1209 - acc: 0.1038 - val_loss: 15.6144 - val_acc: 0.0312

Epoch 00004: val_loss did not improve from 6.05639
Epoch 5/20
39/39 [==============================] - 293s 8s/step - loss: 2.8852 - acc: 0.1458 - val_loss: 15.3626 - val_acc: 0.0469

Epoch 00005: val_loss

In [10]:
#Predict on validation dataset
predictions = model.predict_generator(val_generator, steps=1)        
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

predictions = [label_map[k] for k in predictions]

print(predictions, len(predictions))

loss, acc = model.evaluate_generator(val_generator, steps=1, verbose=0)

print(loss,",",acc)

['H', 'K', 'I', 'W', 'G', 'Q', 'L', 'R', 'F', 'del', 'W', 'K', 'H', 'M', 'R', 'L', 'E', 'P', 'R', 'Q', 'B', 'H', 'C', 'F', 'B', 'del', 'nothing', 'R', 'R', 'Z', 'Z', 'W', 'B', 'K', 'S', 'B', 'E', 'O', 'nothing', 'E', 'Z', 'B', 'nothing', 'W', 'R', 'W', 'Q', 'C', 'W', 'S', 'Q', 'X', 'B', 'P', 'Q', 'W', 'S', 'R', 'nothing', 'Q', 'R', 'Q', 'del', 'B'] 64
4.521135330200195 , 0.34375


In [11]:
test_image_generator = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True,
)

test_generator = test_image_generator.flow_from_directory(TEST_DIR, target_size=TARGET_SIZE, batch_size=28, shuffle=False, 
    class_mode='categorical')
print(test_generator.classes)

Found 28 images belonging to 1 classes.
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [12]:
#Predict
test_generator.reset()
predictions = model.predict_generator(test_generator, steps=1)
predictions = np.argmax(predictions, axis=1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

predictions = [label_map[k] for k in predictions]

print(predictions, len(predictions))

['M', 'B', 'C', 'D', 'E', 'F', 'H', 'H', 'I', 'H', 'K', 'L', 'E', 'H', 'P', 'Q', 'Q', 'R', 'M', 'J', 'U', 'W', 'W', 'R', 'J', 'Z', 'nothing', 'L'] 28


In [13]:
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
print(results)

                              Filename Predictions
0         asl_alphabet_test\A_test.jpg           M
1         asl_alphabet_test\B_test.jpg           B
2         asl_alphabet_test\C_test.jpg           C
3         asl_alphabet_test\D_test.jpg           D
4         asl_alphabet_test\E_test.jpg           E
5         asl_alphabet_test\F_test.jpg           F
6         asl_alphabet_test\G_test.jpg           H
7         asl_alphabet_test\H_test.jpg           H
8         asl_alphabet_test\I_test.jpg           I
9         asl_alphabet_test\J_test.jpg           H
10        asl_alphabet_test\K_test.jpg           K
11        asl_alphabet_test\L_test.jpg           L
12        asl_alphabet_test\M_test.jpg           E
13        asl_alphabet_test\N_test.jpg           H
14        asl_alphabet_test\O_test.jpg           P
15        asl_alphabet_test\P_test.jpg           Q
16        asl_alphabet_test\Q_test.jpg           Q
17        asl_alphabet_test\R_test.jpg           R
18        asl_alphabet_test\S_t

In [14]:
count = 0
for file,prediction in zip(filenames,predictions):
    #print(file,prediction)
    if(prediction+'_test' in file):
        count+=1
        
print("accuracy",count/len(filenames)*100)

accuracy 53.57142857142857


In [15]:
#Define Xception Model with pretrained weight
def Xception_model_build_weight():
    from keras.applications.xception import Xception
    from keras.layers import Input

    input_tensor = Input(shape=TARGET_DIMS)
    model = Xception(input_tensor = input_tensor, weights='imagenet', include_top=False)
    return model

from keras.models import Model
weight_base_model = Xception_model_build_weight()
x1 = weight_base_model.output
#Add the fully-connected layers 
x1 = Flatten(name='flatten')(x1)
x1 = Dense(4096, activation='relu', name='fc1')(x1)
x1 = Dense(4096, activation='relu', name='fc2')(x1)
x1 = Dense(CLASSES, activation='softmax', name='predictions')(x1)
weight_model = Model(inputs=weight_base_model.input, outputs=x1)

#Compile the model
weight_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

weight_model.summary()
MODEL_PATH = MODEL_DIR+"/Model1-xception-withweight.h5"
weight_model.save(MODEL_PATH)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 31, 31, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 31, 31, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 31, 31, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [16]:
#Checkpointer to save the best models
MODEL_WEIGHT_PATH = MODEL_DIR+"/Model_Weight1-xception-withweight.h5"
checkpointer = ModelCheckpoint(filepath=MODEL_WEIGHT_PATH, 
                               verbose=1, save_best_only=True)

steps_per_epoch = int( np.ceil(len(train_generator)*2 / BATCH_SIZE) )
validation_steps = int( np.ceil(len(val_generator)*2 / BATCH_SIZE) )

weight_model.fit_generator(train_generator, validation_data=val_generator, 
                    steps_per_epoch =  steps_per_epoch,
                    validation_steps = validation_steps,
                    epochs=10, callbacks=[checkpointer], verbose=1)

Epoch 1/10
39/39 [==============================] - 339s 9s/step - loss: 3.5081 - acc: 0.0288 - val_loss: 3.3677 - val_acc: 0.0344

Epoch 00001: val_loss improved from inf to 3.36771, saving model to ./Model/Model_Weight1-xception-withweight.h5
Epoch 2/10
39/39 [==============================] - 324s 8s/step - loss: 3.3676 - acc: 0.0333 - val_loss: 3.3724 - val_acc: 0.0316

Epoch 00002: val_loss did not improve from 3.36771
Epoch 3/10
39/39 [==============================] - 328s 8s/step - loss: 3.3685 - acc: 0.0341 - val_loss: 3.3681 - val_acc: 0.0219

Epoch 00003: val_loss did not improve from 3.36771
Epoch 4/10
39/39 [==============================] - 327s 8s/step - loss: 3.3671 - acc: 0.0308 - val_loss: 3.3690 - val_acc: 0.0281

Epoch 00004: val_loss did not improve from 3.36771
Epoch 5/10
39/39 [==============================] - 328s 8s/step - loss: 3.3674 - acc: 0.0417 - val_loss: 3.3671 - val_acc: 0.0375

Epoch 00005: val_loss improved from 3.36771 to 3.36710, saving model to ./

In [17]:
#Predict on validation dataset
predictions = weight_model.predict_generator(val_generator, steps=1)        
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

predictions = [label_map[k] for k in predictions]

print(predictions, len(predictions))

loss, acc = weight_model.evaluate_generator(val_generator, steps=1, verbose=0)

print(loss,",",acc)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] 64
3.3681979179382324 , 0.015625


In [18]:
test_image_generator = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True,
)

test_generator = test_image_generator.flow_from_directory(TEST_DIR, target_size=TARGET_SIZE, batch_size=28, shuffle=False, 
    class_mode='categorical')
print(test_generator.classes)

Found 28 images belonging to 1 classes.
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [19]:
#Predict
test_generator.reset()
predictions = weight_model.predict_generator(test_generator, steps=1)
predictions = np.argmax(predictions, axis=1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

predictions = [label_map[k] for k in predictions]

print(predictions, len(predictions))

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] 28


In [20]:
filenames=test_generator.filenames
count = 0
for file,prediction in zip(filenames,predictions):
    #print(file,prediction)
    if(prediction+'_test' in file):
        count+=1
        
print("accuracy",count/len(filenames)*100)

accuracy 3.571428571428571
